In [1]:
import os
import pathlib
import shutil
import tensorflow as tf
import numpy as np
import cv2
from tensorflow import keras
from keras.models import Sequential
from keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
#Complete dataset path
full_data_dir = 'C:\\Users\\SAHIL\\Driver-Pose-Dataset\\imgs\\train'

In [3]:
#making a path for base directory to work on
base_dir = './IMAGES'
pathlib.Path(base_dir).mkdir(exist_ok=True)

In [4]:
# Train, validation and test data to split and work
train_dir = os.path.join(base_dir, 'Train')
pathlib.Path(train_dir).mkdir(exist_ok=True)

validation_dir = os.path.join(base_dir, 'Validation')
pathlib.Path(validation_dir).mkdir(exist_ok=True)

test_dir = os.path.join(base_dir, 'Test')
pathlib.Path(test_dir).mkdir(exist_ok=True)

In [5]:
#Copying images in the required fashion using manual splitting
dirs = {'Train': train_dir, 'Validation': validation_dir, 'Test': test_dir}

classnames = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']

for (dataset_type, start, end) in [('Train', 0, 1300), ('Validation', 1300, 1600), ('Test', 1600, 1900)]:
    print('Copying ' + dataset_type + ' data')
    for classname in classnames:
        print('Copying ' + classname + ' data')
        pathlib.Path(os.path.join(dirs[dataset_type], classname)).mkdir(exist_ok=True)
        file_names = os.listdir(os.path.join(full_data_dir, classname))[start:end]
        for file_name in file_names:
            src = os.path.join(full_data_dir, classname, file_name)
            dst = os.path.join(dirs[dataset_type], classname, file_name)
            shutil.copyfile(src, dst)

Copying Train data
Copying c0 data
Copying c1 data
Copying c2 data
Copying c3 data
Copying c4 data
Copying c5 data
Copying c6 data
Copying c7 data
Copying c8 data
Copying c9 data
Copying Validation data
Copying c0 data
Copying c1 data
Copying c2 data
Copying c3 data
Copying c4 data
Copying c5 data
Copying c6 data
Copying c7 data
Copying c8 data
Copying c9 data
Copying Test data
Copying c0 data
Copying c1 data
Copying c2 data
Copying c3 data
Copying c4 data
Copying c5 data
Copying c6 data
Copying c7 data
Copying c8 data
Copying c9 data


In [6]:
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
#Sequential model - CNN

model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(230, 230, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))


In [8]:
#compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['acc'])

In [9]:
#generators for our model
train_generator = train_datagen.flow_from_directory(
        train_dir,
        # All images will be resized to 230x230
        target_size=(230, 230),
        batch_size=50,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(230, 230),
        batch_size=50,
        class_mode='categorical')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=260,
      epochs=9,
      verbose = 1,
      validation_data=validation_generator,
      validation_steps=60)


Found 13000 images belonging to 10 classes.
Found 3000 images belonging to 10 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 260 steps, validate for 60 steps
Epoch 1/9
260/260 [==============================] - 622s 2s/step - loss: 1.2866 - acc: 0.5426 - val_loss: 0.4360 - val_acc: 0.8743
Epoch 2/9
260/260 [==============================] - 660s 3s/step - loss: 0.3492 - acc: 0.8929 - val_loss: 0.1885 - val_acc: 0.9443
Epoch 3/9
260/260 [==============================] - 1763s 7s/step - loss: 0.1847 - acc: 0.9427 - val_loss: 0.1032 - val_acc: 0.9700
Epoch 4/9
260/260 [==============================] - 637s 2s/step - loss: 0.1326 - acc: 0.9594 - val_loss: 0.0775 - val_acc: 0.9793
Epoch 5/9
260/260 [==============================] - 631s 2s/step - loss: 0.0897 - acc: 0.9717 - val_loss: 0.0777 - val_acc: 0.9807
Epoch 6/9
260/260 [==============================] - 622s 2s/step - loss: 0.0736 - 

In [10]:
#Saving the model
model.save('Model.h5')

In [11]:
model = tf.keras.models.load_model('Model.h5')

In [12]:
#evaluating on test data
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(230, 230),
        batch_size=50,
        class_mode='categorical')

test_loss, test_acc = model.evaluate(test_generator, steps=60)
print('test_loss acc:', test_loss)
print('test acc:', test_acc)

Found 3000 images belonging to 10 classes.
  ...
    to  
  ['...']
60/60 [==============================] - 35s 585ms/step - loss: 0.0587 - acc: 0.9883
test_loss acc: 0.05866525627983113
test acc: 0.98833334


In [13]:
#class labels
class_labels = [
    "Normal Driving",
    "Texting - Right Hand",
    "Talking on the phone - Right Hand",
    "Texting - Left Hand",
    "Talking on the phone - Left Hand",
    "Operating the Radio",
    "Drinking",
    "Reaching behind",
    "Hair and makeup",
    "Talking to passenger"
]

In [14]:
#testing on an image manually selected by the user
test_image_path = "./img_100038.jpg"

img_array = []

img = image.load_img(test_image_path, target_size=(230, 230))
img_array = image.img_to_array(img)
img_array = np.array([img_array])

img_array = img_array.astype('float32') / 255
predictions = model.predict(img_array)

label_index = int(np.argmax(predictions, axis = 1))
print(label_index)

3


In [15]:
#showing the output using openCV
test_image = cv2.imread(test_image_path)

cv2.putText(test_image, class_labels[label_index], (320, 45), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
cv2.imshow("Image",test_image)
    
cv2.waitKey(0)
    
cv2.destroyAllWindows()